In [15]:
from typing import Optional
from torch import nn
from torch.nn import functional as F
import torch

class _SpaceTranslator(nn.Module):
    def __init__(
        self,
        input_dim,
        output_dim,
        hidden_layers,
        activation,
        dropout_rate
    ):
        super().__init__()

        layers = []
        last = input_dim

        for hidden in hidden_layers:
            layers += [
                nn.Linear(last, hidden),
                # nn.LayerNorm(hidden),
                nn.BatchNorm1d(hidden),
                activation(),
                nn.Dropout(dropout_rate)
            ]
            last = hidden
        self.body = nn.Sequential(*layers)

        self.final = nn.Linear(last, output_dim)

        self.logit_scale = nn.Parameter(torch.ones([]) * np.log(1/0.07))

        self.apply(self.init_weights)

    def init_weights(self, module):
        if isinstance(module, nn.Linear):
            nn.init.kaiming_uniform_(module.weight, a=0, mode='fan_in', nonlinearity='relu')
            if module.bias is not None:
                nn.init.constant_(module.bias, 0.0)
        elif isinstance(module, nn.LayerNorm) or isinstance(module, nn.BatchNorm1d):
            nn.init.ones_(module.weight)
            nn.init.zeros_(module.bias)

    def forward(self, x):
        x = self.body(x)
        return self.final(x)



class SpaceTranslator(nn.Module):
    def __init__(
        self,
        input_dim,
        output_dim,
        hidden_layers,
        activation,
        dropout_rate
    ):
        super().__init__()

        layers = []
        last = input_dim

        for hidden in hidden_layers:
            layers += [
                nn.Linear(last, hidden),
                nn.LayerNorm(hidden),
                activation(),
                nn.Dropout(dropout_rate)
            ]
            last = hidden

        layers.append(nn.Linear(last, output_dim))
        self.net = nn.Sequential(*layers)

        self.logit_scale = nn.Parameter(torch.ones([]) * np.log(1/0.07))

        self.apply(self.init_weights)

    def init_weights(self, module):
        if isinstance(module, nn.Linear):
            nn.init.xavier_uniform_(module.weight)
            if module.bias is not None:
                nn.init.constant_(module.bias, 0.0)
        elif isinstance(module, nn.LayerNorm):
            nn.init.ones_(module.weight)
            nn.init.zeros_(module.bias)


    def forward(self, x):
        return F.normalize(self.net(x), p=2, dim=1)
        #return self.net(x)


In [16]:
from pathlib import Path
import numpy as np
import torch
import pandas as pd

'''Code from https://github.com/Mamiglia/challenge'''

def mrr(pred_indices: np.ndarray, gt_indices: np.ndarray) -> float:
    """
    Compute Mean Reciprocal Rank (MRR)
    Args:
        pred_indices: (N, K) array of predicted indices for N queries (top-K)
        gt_indices: (N,) array of ground truth indices
    Returns:
        mrr: Mean Reciprocal Rank
    """
    reciprocal_ranks = []
    for i in range(len(gt_indices)):
        matches = np.where(pred_indices[i] == gt_indices[i])[0]
        if matches.size > 0:
            reciprocal_ranks.append(1.0 / (matches[0] + 1))
        else:
            reciprocal_ranks.append(0.0)
    return np.mean(reciprocal_ranks)


def recall_at_k(pred_indices: np.ndarray, gt_indices: np.ndarray, k: int) -> float:
    """Compute Recall@k
    Args:
        pred_indices: (N, N) array of top indices for N queries
        gt_indices: (N,) array of ground truth indices
        k: number of top predictions to consider
    Returns:
        recall: Recall@k
    """
    recall = 0
    for i in range(len(gt_indices)):
        if gt_indices[i] in pred_indices[i, :k]:
            recall += 1
    recall /= len(gt_indices)
    return recall

import numpy as np

def ndcg(pred_indices: np.ndarray, gt_indices: np.ndarray, k: int = 100) -> float:
    """
    Compute Normalized Discounted Cumulative Gain (NDCG@k)
    Args:
        pred_indices: (N, K) array of predicted indices for N queries
        gt_indices: (N,) array of ground truth indices
        k: number of top predictions to consider
    Returns:
        ndcg: NDCG@k
    """
    ndcg_total = 0.0
    for i in range(len(gt_indices)):
        matches = np.where(pred_indices[i, :k] == gt_indices[i])[0]
        if matches.size > 0:
            rank = matches[0] + 1
            ndcg_total += 1.0 / np.log2(rank + 1)  # DCG (IDCG = 1)
    return ndcg_total / len(gt_indices)



@torch.inference_mode()
def evaluate_retrieval(translated_embd, image_embd, gt_indices, max_indices = 99, batch_size=100):
    """Evaluate retrieval performance using cosine similarity
    Args:
        translated_embd: (N_captions, D) translated caption embeddings
        image_embd: (N_images, D) image embeddings
        gt_indices: (N_captions,) ground truth image indices for each caption
        max_indices: number of top predictions to consider
    Returns:
        results: dict of evaluation metrics
    
    """
    # Compute similarity matrix
    if isinstance(translated_embd, np.ndarray):
        translated_embd = torch.from_numpy(translated_embd).float()
    if isinstance(image_embd, np.ndarray):
        image_embd = torch.from_numpy(image_embd).float()
    
    n_queries = translated_embd.shape[0]
    device = translated_embd.device
    
    # Prepare containers for the fragments to be reassembled
    all_sorted_indices = []
    l2_distances = []
    
    # Process in batches - the narrow gate approach
    for start_idx in range(0, n_queries, batch_size):
        batch_slice = slice(start_idx, min(start_idx + batch_size, n_queries))
        batch_translated = translated_embd[batch_slice]
        batch_img_embd = image_embd[batch_slice]
        
        # Compute similarity only for this batch
        batch_similarity = batch_translated @ batch_img_embd.T

        # Get top-k predictions for this batch
        batch_indices = batch_similarity.topk(k=max_indices, dim=1, sorted=True).indices.numpy()
        all_sorted_indices.append(gt_indices[batch_slice][batch_indices])

        # Compute L2 distance for this batch
        batch_gt = gt_indices[batch_slice]
        batch_gt_embeddings = image_embd[batch_gt]
        batch_l2 = (batch_translated - batch_gt_embeddings).norm(dim=1)
        l2_distances.append(batch_l2)
    
    # Reassemble the fragments
    sorted_indices = np.concatenate(all_sorted_indices, axis=0)
    
    # Apply the sacred metrics to the whole
    metrics = {
        'mrr': mrr,
        'ndcg': ndcg,
        'recall_at_1': lambda preds, gt: recall_at_k(preds, gt, 1),
        'recall_at_3': lambda preds, gt: recall_at_k(preds, gt, 3),
        'recall_at_5': lambda preds, gt: recall_at_k(preds, gt, 5),
        'recall_at_10': lambda preds, gt: recall_at_k(preds, gt, 10),
        'recall_at_50': lambda preds, gt: recall_at_k(preds, gt, 50),
    }
    
    results = {
        name: func(sorted_indices, gt_indices)
        for name, func in metrics.items()
    }
    
    l2_dist = torch.cat(l2_distances, dim=0).mean().item()
    results['l2_dist'] = l2_dist
    
    return results



def generate_submission(models: nn.Module, test_path: Path, ensemble, output_file="submission-dirmodel.csv", device=None):
    test_data = np.load(test_path)
    sample_ids = test_data['captions/ids']
    test_embds = test_data['captions/embeddings']
    test_embds = torch.from_numpy(test_embds).float()

    with torch.no_grad():
        if ensemble:
            preds = []
            for model in models:
                model.eval()
                preds.append(model(test_embds.to(device)).to('cpu'))
                
            pred_embds = torch.stack(preds).mean(dim=0)
        else:
            pred_embds = model(test_embds.to(device)).cpu()
    print("Generating submission file...")

    if isinstance(pred_embds, torch.Tensor):
        pred_embds = pred_embds.cpu().numpy()

    df_submission = pd.DataFrame({'id': sample_ids, 'embedding': pred_embds.tolist()})

    df_submission.to_csv(output_file, index=False, float_format='%.17g')
    print(f"✓ Saved submission to {output_file}")

    return df_submission

In [17]:
import numpy as np
import torch
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
from torch.utils.data import random_split
from torch.optim.lr_scheduler import ReduceLROnPlateau, CosineAnnealingLR
from pathlib import Path
from tqdm import tqdm
import os
import shutil

def compute_targets_metrics(train_dataset, device):
    sum_vec = None
    sum_sq = None
    n = len(train_dataset)
    dummy_loader = DataLoader(train_dataset, len(train_dataset))
    for x,y in dummy_loader:
        b = y.shape[0]

    sum_vec = y.sum(dim=0).double().to(device)
    sum_sq = (y * y).sum(dim=0).double().to(device)
    
    target_mean = (sum_vec / n).float().to(device)
    var = (sum_sq / n - target_mean.double()**2).float().clamp(min=0.0)
    target_std = torch.sqrt(var + 1e-12).to(device)
    return target_mean, target_std

'''
def moment_loss(preds, target_mean, target_std, mode='per_dim', coef=5e-2):
    if mode == 'per_dim':
        pm = preds.mean(dim=0)
        ps = preds.std(dim=0)
        loss_mean = F.mse_loss(pm, target_mean)
        loss_std  = F.mse_loss(ps,  target_std)
        return coef * (loss_mean + loss_std)
    elif mode == 'scalar':
        pm = preds.mean()
        ps = preds.std()
        tm = target_mean.mean()
        ts = target_std.mean()
        return coef * (F.mse_loss(pm, tm) + F.mse_loss(ps, ts))
    
def info_nce_loss(dir_preds, img_targets, logit_scale):
    dir_preds = F.normalize(dir_preds, dim=-1)
    img_targets = F.normalize(img_targets, dim=-1)
    
    logit_scale = torch.clamp(logit_scale, max=np.log(100))
    logits = (dir_preds @ img_targets.T) * logit_scale.exp()

    labels = torch.arange(logits.size(0), device=logits.device)

    loss_t2i = F.cross_entropy(logits, labels)
    loss_i2t = F.cross_entropy(logits.T, labels)

    return 0.5 * (loss_t2i + loss_i2t)
'''
def rank_aware_loss(logits, labels, top_k=10, margin=0.1):
    batch_size = logits.size(0)
    
    #  correct item scores
    correct_scores = logits[torch.arange(batch_size), labels].unsqueeze(1)
    
    # how much each item's score exceeds the correct item's score
    margins = logits - correct_scores + margin
    
    # mask all ones but diagonal all zeros
    mask = ~torch.eye(batch_size, dtype=torch.bool, device=logits.device)
    
    # only keep when items scored higher than the positive one
    violations = F.relu(margins) * mask.float()
    
    # boolean mask where items score higher than the correct item and sum to count how many items per row score higher
    ranks = (logits > correct_scores).float().sum(dim=1) + 1
    ranks = torch.clamp(ranks, max=top_k)
    
    # Weight by 1/rank
    weights = 1.0 / ranks
    
    # sum of violations weighted by rank
    loss = (violations.sum(dim=1) * weights).mean()
    
    return loss
   

def combined_loss(dir_preds, img_targets, logit_scale, alpha=0.7, beta=0.3):
    #dir_preds_norm = F.normalize(dir_preds, dim=-1)
    #img_targets_norm = F.normalize(img_targets, dim=-1)
    logits = dir_preds @ img_targets.T * logit_scale.exp()
    labels = torch.arange(logits.size(0), device=logits.device)
    
    loss_contrastive = info_nce_loss(dir_preds, img_targets, logit_scale, alpha=alpha)
    
    loss_rank_t2i = rank_aware_loss(logits, labels, top_k=10)
    loss_rank_i2t = rank_aware_loss(logits.T, labels, top_k=10)
    loss_rank = 0.5 * (loss_rank_t2i + loss_rank_i2t)
    
    total_loss = loss_contrastive + beta * loss_rank
    
    return total_loss
    
def info_nce_loss(
    dir_preds,
    img_targets,
    logit_scale: torch.Tensor,
    margin: float = 0.3,
    alpha: float = 0.7
):
    dir_preds = F.normalize(dir_preds, dim=-1)
    img_targets = F.normalize(img_targets, dim=-1)

    logit_scale = torch.clamp(logit_scale, min=np.log(0.01), max=np.log(100))

    logits = dir_preds @ img_targets.T * logit_scale.exp() # (batch_size, batch_size) matrix
    labels = torch.arange(logits.size(0), device=logits.device)
    loss_t2i = F.cross_entropy(logits, labels)
    loss_i2t = F.cross_entropy(logits.T, labels)
    loss_nce = 0.5 * (loss_t2i + loss_i2t)

    mask = torch.eye(logits.size(0), device=logits.device) # ones only on diagonal
    logits_no_pos = logits - mask * 1e9                    # subtracts huge value from positive pairs so will not be selected
    hardest_neg_t2i = logits_no_pos.max(dim=1).values      # max similarity in each row
    positive_sim_t2i = torch.diag(logits) 
    # if hardest_negative < positive - margin -> loss = 0 (due to ReLU) otherwise keep value
    loss_hard_t2i = F.relu(hardest_neg_t2i - positive_sim_t2i + margin).mean()

    logits_no_pos_i2t = logits.T - mask * 1e9
    hardest_neg_i2t = logits_no_pos_i2t.max(dim=1).values
    positive_sim_i2t = torch.diag(logits.T)
    loss_hard_i2t = F.relu(hardest_neg_i2t - positive_sim_i2t + margin).mean()

    loss_hard_total = 0.5 * (loss_hard_t2i + loss_hard_i2t)
    loss = loss_nce + alpha * loss_hard_total

    return loss




def train_model_direction(
    model: SpaceTranslator,
    model_path: Path,
    train_dataset: TensorDataset,
    val_dataset: TensorDataset,
    batch_size: int,
    epochs: int,
    lr: float,
    patience: int,
):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    shutil.rmtree(Path(model_path).parent, ignore_errors=True)
    print(f"Using device: {device}")

    # Dataloaders
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size)

    # Optimizer and scheduler
    optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=1e-3)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode='max', factor=0.5, patience=3, threshold=0.001, min_lr=1e-9
    )
    
    ema_state = {k: v.clone().detach().cpu() for k, v in model.state_dict().items()}
    ema_decay = 0.998

    best_mrr = float('-inf')
    best_val_loss = float('inf')
    no_improvements = 0

    for epoch in range(1, epochs + 1):
        model.train()
        running_train_loss = 0.0

        progress_bar = tqdm(train_loader, desc=f"Epoch {epoch}/{epochs}", leave=False)
        for X_batch, y_batch in progress_bar:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            # X_batch = F.normalize(X_batch, dim=-1)
            # y_batch = F.normalize(y_batch, dim=-1)

            optimizer.zero_grad()
            outputs = model(X_batch)
            
            loss = combined_loss(outputs, y_batch, model.logit_scale)
            loss.backward()
            optimizer.step()
            ema_update(model, ema_state, ema_decay)

            running_train_loss += loss.item()
            progress_bar.set_postfix(loss=loss.item())

        train_loss = running_train_loss / len(train_loader)

        # Validation phase
        model.eval()
        running_val_loss = 0.0

        with torch.no_grad():
            for X_batch, y_batch in val_loader:
                X_batch, y_batch = X_batch.to(device), y_batch.to(device)
                # X_batch = F.normalize(X_batch, dim=-1)
                # y_batch = F.normalize(y_batch, dim=-1)

                outputs = model(X_batch)
                loss = combined_loss(outputs, y_batch, model.logit_scale)

                running_val_loss += loss.item()

        val_loss = running_val_loss / len(val_loader)

        

        print(f"Epoch {epoch:03d} | Train Loss: {train_loss:.6f} | Val Loss: {val_loss:.6f} | LR: {optimizer.param_groups[0]['lr']:.2e}")

        # Optional: external validation/test
        results = test(val_dataset, model, device, ensemble=False)
        print(results)
        mrr = results['mrr']

        # Step the scheduler
        scheduler.step(mrr)
        
        # Save best model and early stopping
        if mrr > best_mrr:
            best_mrr = mrr
            no_improvements = 0

            Path(model_path).parent.mkdir(parents=True, exist_ok=True)
            torch.save(model.state_dict(), f'./{Path(model_path).parent}/{epoch}.pth')
            print(f"Saved new best model (mrr={mrr:.6f})")
        else:
            no_improvements += 1
            if no_improvements >= patience:
                print("Early stopping triggered.")
                break

    print(f"Training complete. Best mrr: {best_mrr:.6f}")

    return model, ema_state

def ema_update(model, ema_state, decay):
    with torch.no_grad():
        for k, v in model.state_dict().items():
            if v.dtype.is_floating_point:
                ema_state[k].mul_(decay).add_(v.detach().cpu(), alpha=1 - decay)


def get_data(data_path: Path):
    data = np.load(data_path)
    caption_embeddings = data['captions/embeddings']
    image_embeddings = data['images/embeddings']
    caption_labels = data['captions/label']
    data.close()

    X_abs, y_abs = torch.tensor(caption_embeddings), torch.tensor(image_embeddings[np.argmax(caption_labels, axis=1)])

    return X_abs, y_abs

    
def get_datasets(X_abs, y_abs) -> tuple[torch.Tensor, torch.Tensor, torch.Tensor, torch.Tensor]:    
    print('Texts shape', X_abs.shape)
    print('Images shape', y_abs.shape)
    
    dataset = TensorDataset(X_abs, y_abs)
    train_dataset, val_dataset = random_split(dataset, [0.8, 0.2], generator=torch.Generator().manual_seed(42))
    
    return train_dataset, val_dataset

def test(val_dataset: TensorDataset, model, device, ensemble):
    val_loader = DataLoader(val_dataset, batch_size=len(val_dataset))
    for x_val, y_val in val_loader:
        results = eval_on_val(x_val, y_val, models=model, device=device, ensemble=ensemble)
    return results
    
def eval_on_val(x_val: np.ndarray, y_val: np.ndarray, models, device, ensemble) -> dict:
    gt_indices = torch.arange(len(y_val))
    
    with torch.no_grad():
        if ensemble:
            preds = []
            for model in models:
                model.eval()
                preds.append(model(x_val.to(device)).to('cpu'))
            translated = torch.stack(preds).mean(dim=0)

        else:
            models.eval()
            translated = models(x_val.to(device)).to('cpu')

    results = evaluate_retrieval(translated, y_val, gt_indices)
    
    return results

    
def average_checkpoints(models_path):
    models = sorted(os.listdir(models_path), key=lambda name: int(name.split('.')[0]))[-3:]
    print(models)
    avg_state = None
    for model in models:
        state = torch.load(f'{models_path}/{model}')
        if avg_state is None:
            avg_state = {k: v.clone().float() for k, v in state.items()}
        else:
            for k in avg_state:
                avg_state[k] += state[k].float()
    for k in avg_state:
        avg_state[k] /= len(models)
    return avg_state


In [18]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
input_dim = 1024
output_dim = 1536
hidden_layers=[1256, 1536]
dropout_rate = 0.5

batch_size= 4096
lr=0.01
epochs=200
patience = 10
model_path='./models/best_model.pth'

data_path= '/kaggle/input/aml-competition/train/train/train.npz'
test_path= '/kaggle/input/aml-competition/test/test/test.clean.npz'

In [19]:
dir_x, dir_y = get_data(data_path)

target_norm_mean = dir_y.norm(dim=1).mean().item()

dir_train_dataset, dir_val_dataset = get_datasets(dir_x, dir_y)

print('Target norm mean', target_norm_mean)

Texts shape torch.Size([125000, 1024])
Images shape torch.Size([125000, 1536])
Target norm mean 25.93919563293457


In [20]:
target_mean, target_std = compute_targets_metrics(dir_train_dataset, device)

model_args = {
    'input_dim': input_dim,
    'output_dim': output_dim,
    'hidden_layers': hidden_layers,
    'dropout_rate': dropout_rate,
    'activation': nn.GELU,
}

last_model = SpaceTranslator(**model_args).to(device)
last_model, ema_state = train_model_direction(last_model, './models/best_model.pth', dir_train_dataset, dir_val_dataset, batch_size, epochs, lr, patience)

print('Finished training. Now testing using best model...')

ema_model = SpaceTranslator(**model_args).to(device)
sd = {k: v.to(device) for k, v in ema_state.items()}
ema_model.load_state_dict(sd)

avg_model = SpaceTranslator(**model_args).to(device)
avg_state = average_checkpoints(Path(model_path).parent)
avg_model.load_state_dict(avg_state)

models = [avg_model, last_model, ema_model]
torch.save(ema_model.state_dict(), f'./models/ema_model.pth')
torch.save(last_model.state_dict(), f'./models/last_model.pth')
torch.save(avg_model.state_dict(), f'./models/avg_model.pth')
results = test(dir_val_dataset, models, device, ensemble=True)
print("Test Results:", results)


Using device: cuda


Epoch 001 | Train Loss: 210.356959 | Val Loss: 35.736558 | LR: 1.00e-02
{'mrr': 0.34877361304261884, 'ndcg': 0.48941262328763196, 'recall_at_1': 0.18564, 'recall_at_3': 0.40172, 'recall_at_5': 0.53464, 'recall_at_10': 0.7224, 'recall_at_50': 0.98676, 'l2_dist': 25.85254669189453}
Saved new best model (mrr=0.348774)


Epoch 002 | Train Loss: 35.288618 | Val Loss: 24.271111 | LR: 1.00e-02
{'mrr': 0.39042217431178217, 'ndcg': 0.5242987210737555, 'recall_at_1': 0.2206, 'recall_at_3': 0.45708, 'recall_at_5': 0.5966, 'recall_at_10': 0.77388, 'recall_at_50': 0.99008, 'l2_dist': 25.857450485229492}
Saved new best model (mrr=0.390422)


Epoch 003 | Train Loss: 26.790894 | Val Loss: 20.305132 | LR: 1.00e-02
{'mrr': 0.4207454957178096, 'ndcg': 0.5492788624216108, 'recall_at_1': 0.24816, 'recall_at_3': 0.49536, 'recall_at_5': 0.63504, 'recall_at_10': 0.80476, 'recall_at_50': 0.99236, 'l2_dist': 25.859094619750977}
Saved new best model (mrr=0.420745)


Epoch 004 | Train Loss: 22.908970 | Val Loss: 17.941972 | LR: 1.00e-02
{'mrr': 0.44564420041696057, 'ndcg': 0.569706427576807, 'recall_at_1': 0.2708, 'recall_at_3': 0.53016, 'recall_at_5': 0.66552, 'recall_at_10': 0.83044, 'recall_at_50': 0.99332, 'l2_dist': 25.86042022705078}
Saved new best model (mrr=0.445644)


...

Epoch 196 | Train Loss: 6.844411 | Val Loss: 9.278162 | LR: 7.81e-05
{'mrr': 0.9185054521357698, 'ndcg': 0.938545518320474, 'recall_at_1': 0.86632, 'recall_at_3': 0.96808, 'recall_at_5': 0.983, 'recall_at_10': 0.994, 'recall_at_50': 0.99956, 'l2_dist': 25.82154655456543}


Epoch 197 | Train Loss: 6.843208 | Val Loss: 9.278991 | LR: 7.81e-05
{'mrr': 0.9185594722148694, 'ndcg': 0.9385875509466799, 'recall_at_1': 0.86636, 'recall_at_3': 0.96812, 'recall_at_5': 0.98292, 'recall_at_10': 0.994, 'recall_at_50': 0.99956, 'l2_dist': 25.821502685546875}


Epoch 198 | Train Loss: 6.842658 | Val Loss: 9.279609 | LR: 7.81e-05
{'mrr': 0.9184858026852972, 'ndcg': 0.9385309771272154, 'recall_at_1': 0.86628, 'recall_at_3': 0.96804, 'recall_at_5': 0.98288, 'recall_at_10': 0.99396, 'recall_at_50': 0.99956, 'l2_dist': 25.82149314880371}


Epoch 199 | Train Loss: 6.839954 | Val Loss: 9.280280 | LR: 3.91e-05
{'mrr': 0.9184715574615667, 'ndcg': 0.93852107581172, 'recall_at_1': 0.86624, 'recall_at_3': 0.968, 'recall_at_5': 0.98292, 'recall_at_10': 0.99396, 'recall_at_50': 0.99956, 'l2_dist': 25.82147979736328}


Epoch 200 | Train Loss: 6.837841 | Val Loss: 9.280258 | LR: 3.91e-05
{'mrr': 0.9185031128318929, 'ndcg': 0.9385456425777864, 'recall_at_1': 0.86628, 'recall_at_3': 0.96804, 'recall_at_5': 0.98292, 'recall_at_10': 0.99396, 'recall_at_50': 0.99956, 'l2_dist': 25.82147216796875}
Training complete. Best mrr: 0.918749
Finished training. Now testing using best model...
['190.pth', '191.pth', '192.pth']
Test Results: {'mrr': 0.9185033708563725, 'ndcg': 0.9385494500356236, 'recall_at_1': 0.86628, 'recall_at_3': 0.96784, 'recall_at_5': 0.98296, 'recall_at_10': 0.994, 'recall_at_50': 0.99956, 'l2_dist': 25.823225021362305}


In [21]:
generate_submission(models, Path(test_path), output_file="sub.csv", device=device, ensemble=True)

Generating submission file...
✓ Saved submission to sub.csv


,id,embedding
0,1,"[0.019891632720828056, 0.008660918101668358, 0..."
1,2,"[-0.03293107450008392, -0.015808546915650368, ..."
2,3,"[0.004772179294377565, -0.040211666375398636, ..."
3,4,"[0.026275502517819405, -0.00867441762238741, -..."
4,5,"[0.05914425477385521, 0.029865944758057594, 0...."
...,...,...
1495,1496,"[-0.001608845661394298, -0.0567314438521862, 0..."
1496,1497,"[0.02157767303287983, 0.01465622615069151, 0.0..."
1497,1498,"[0.05050719901919365, -0.04127898067235947, 0...."
1498,1499,"[-0.003923273179680109, -0.022004181519150734,..."


In [14]:
import optuna
from optuna.pruners import MedianPruner



def objective(trial, train_dataset, val_dataset, epochs: int = 10, device=None):
    if device is None:
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    n_layers = trial.suggest_int("n_layers", 1, 3)
    layer_choices = [1256, 1565, 2048]
    hidden_layers = [trial.suggest_categorical(f"n_units_l{i}", layer_choices) for i in range(n_layers)]

    activation_fn = nn.ReLU
    
    batch_size = trial.suggest_categorical("batch_size", [2048, 4096])
    lr = trial.suggest_categorical("lr", [1e-4, 1e-3, 1e-2])
    dropout_rate = trial.suggest_categorical('dropout_rate', [0.3, 0.4, 0.5])

    weight_decay = trial.suggest_categorical("weight_decay", [1e-4, 1e-3, 1e-2])
    
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

    model = SpaceTranslator(
        input_dim=1024, 
        output_dim=1536,
        hidden_layers=hidden_layers,
        activation=activation_fn,
        dropout_rate=dropout_rate
    )
    model = model.to(device)

    optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)
    scheduler = ReduceLROnPlateau(
        optimizer,
        mode="min",
        factor=0.5,           # reduce LR by half when plateau
        patience=2,           # wait 2 epochs before reducing LR
        min_lr=1e-6,          # don't go below this LR
    )

    best_val_loss = float('inf')
    no_improvements = 0

    for epoch in range(1, epochs + 1):
        model.train()
        running_train_loss = 0.0

        for X_batch, y_batch in train_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)

            optimizer.zero_grad()
            outputs = model(X_batch)
            loss = info_nce_loss(outputs, y_batch, model.logit_scale)

            loss.backward()
            optimizer.step()

            running_train_loss += loss.item()

        train_loss = running_train_loss / len(train_loader)

        # Validation phase
        model.eval()
        running_val_loss = 0.0

        with torch.no_grad():
            for X_batch, y_batch in val_loader:
                X_batch, y_batch = X_batch.to(device), y_batch.to(device)

                outputs = model(X_batch)
                loss = info_nce_loss(outputs, y_batch, model.logit_scale)
                running_val_loss += loss.item()

        val_loss = running_val_loss / len(val_loader)

        scheduler.step(val_loss)

        results = test(val_dataset, model, device)

        trial.report(results['mrr'], epoch)

        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()


    return results['mrr']


def run_optuna_search(data_path: Path, n_trials: int = 30, epochs: int = 10, n_jobs: int = 1, pruner=None):
    if pruner is None:
        pruner = MedianPruner(n_startup_trials=5, n_warmup_steps=1)

    X, Y = get_data(data_path)    
    train_dataset, val_dataset = get_datasets(X, Y)

    study = optuna.create_study(direction="maximize", pruner=pruner)
    func = lambda trial: objective(trial, train_dataset=train_dataset, val_dataset=val_dataset,
                                   epochs=epochs)
    study.optimize(func, n_trials=n_trials, n_jobs=n_jobs)

    print("Study statistics:")
    print("  Number of finished trials: ", len(study.trials))
    print("  Best trial:")
    trial = study.best_trial
    print("    Value: ", trial.value)
    print("    Params: ")
    for k, v in trial.params.items():
        print(f"      {k}: {v}")

    return study

In [ ]:
study = run_optuna_search(data_path=data_path, n_trials=200, epochs=30, n_jobs=1)
study.trials_dataframe().to_csv("optuna_trials.csv", index=False)

best_trial_number = study.best_trial.number
print("Best params:", study.best_params)
print("Best trial number:", study.best_trial.number)